This notebook aims to filter poor quality data to prepare data for actual evapotranspiration estimation from eddy covariance data and also filter 

In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
from scipy.stats import iqr
import copy

## Functions

In [ ]:
# implement energy balance correction for hlaf hourly data and see the results
def EBC_half_hourly_data(half_hourly_filtered_df,
                         flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'}):

   # Ensure index is in datetime format
    df = half_hourly_filtered_df
    df = df.sort_index()
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    df['time_of_day'] = df['hour'] + df['minute'] / 60  # Convert to decimal hour

    # Step 1: Compute EBC_CF for all half-hourly values
    df['EBC_CF'] = (df[flux_cols_dict['Rn']] - df[flux_cols_dict['G']]) / (df[flux_cols_dict['H']] + df[flux_cols_dict['LE']])

    # Step 2: Remove Outliers (Beyond 1.5× IQR)
    Q1, Q3 = df['EBC_CF'].quantile([0.25, 0.75])
    IQR_value = Q3 - Q1
    df['EBC_CF'] = np.where(
        (df['EBC_CF'] < (Q1 - 1.5 * IQR_value)) | (df['EBC_CF'] > (Q3 + 1.5 * IQR_value)),
        np.nan,
        df['EBC_CF']
    )
    
    # Step 3: Compute Smoothed EBC_CF Over ±15 Days for Each Half-Hour Time Step (Method 1)
    def compute_EBC_CF_Method1(timestamp):
        
        """Computes median of values within ±15-day window, filtering between 10:00-14:30 and 22:00-02:30."""
        df_window = df.loc[timestamp - pd.Timedelta(days=15) : timestamp + pd.Timedelta(days=15)]

        # Select values in the two time ranges
        valid_values = pd.concat([
            df_window.between_time("10:00", "14:30"),
            df_window.between_time("22:00", "02:30")
        ])["EBC_CF"].dropna()

        return valid_values.median() if len(valid_values) >= 5 else np.nan
    
    df["EBC_CF_Method1"] = df.index.to_series().apply(compute_EBC_CF_Method1)

    # Step 4: Use Method 2 when "EBC_CF_Method1" is np.nan
    def compute_EBC_CF_Method2(timestamp):
        """ Function to compute Method 2 where Method 1 fails"""
        return (df.loc[timestamp - pd.Timedelta(days=5) : timestamp + pd.Timedelta(days=5)]
                  .between_time("10:00", "14:30")
                  .between_time((timestamp - pd.Timedelta(hours=1)).strftime("%H:%M"),
                                (timestamp + pd.Timedelta(hours=1)).strftime("%H:%M"))["EBC_CF"]
                  .dropna()
                  .mean() if len(df) >= 5 else np.nan)

    df["EBC_CF_Method2"] = df[df["EBC_CF_Method1"].isna()].index.to_series().apply(compute_EBC_CF_Method2)
    
    # Step 5: Final EBC_CF selection: Use Method 1 where available, otherwise use Method 2
    df["EBC_CF_Final"] = df["EBC_CF_Method1"].combine_first(df["EBC_CF_Method2"])
    
    # Step 6: Apply the Corrected EBC_CF to All Data Points
    df['H_corr'] = df[flux_cols_dict['H']] * df['EBC_CF_Final']
    df['LE_corr'] = df[flux_cols_dict['LE']] * df['EBC_CF_Final']

    # Compute Corrected Energy Imbalance (Should be Close to Zero)
    df['Imb_corr'] = df[flux_cols_dict['Rn']] - df[flux_cols_dict['G']] - (df['H_corr'] + df['LE_corr'])

    return df

## Code

#### Preprocess half hourly data

In [ ]:
# read the half hourly data for all the seasons (3 towers with 9 seasons). The data can be downloaded from Ameriflux website!
USUC1_path = os.path.join(os.getcwd(), 'AMF_US-UC1_BASE_HH_5-5.csv')
USUC2_path = os.path.join(os.getcwd(), 'AMF_US-UC2_BASE_HH_5-5.csv')
USHWB_path = os.path.join(os.getcwd(), 'AMF_US-HWB_BASE_HH_2-5.csv')

USUC1_df = pd.read_csv(USUC1_path,skiprows=[0,1] ,index_col = 'TIMESTAMP_START', parse_dates=['TIMESTAMP_START'])
USUC2_df = pd.read_csv(USUC2_path,skiprows=[0,1] ,index_col = 'TIMESTAMP_START', parse_dates=['TIMESTAMP_START'])
USHWB_df = pd.read_csv(USHWB_path,skiprows=[0,1] ,index_col = 'TIMESTAMP_START', parse_dates=['TIMESTAMP_START'])

In [10]:
# for each half hourly data check the flags and drop those rows with LE flag of 2
Rn_col = 'NETRAD_1_1_1'
LE_col = 'LE'
H_col = 'H'
LE_QC_flag_col = 'LE_SSITC_TEST'
H_QC_flag_col = 'H_SSITC_TEST'

# keep only LE values when both LE and H flags are not 2
USUC1_df[LE_col] = USUC1_df.apply(lambda row: row[LE_col] if (row[LE_QC_flag_col]==0) & (row[H_QC_flag_col]==0) else np.nan, axis=1)
USUC2_df[LE_col] = USUC2_df.apply(lambda row: row[LE_col] if (row[LE_QC_flag_col]==0) & (row[H_QC_flag_col]==0) else np.nan, axis=1)
USHWB_df[LE_col] = USHWB_df.apply(lambda row: row[LE_col] if (row[LE_QC_flag_col]==0) & (row[H_QC_flag_col]==0) else np.nan, axis=1)

# keep only H values when both LE and H flags are not 2
USUC1_df[H_col] = USUC1_df.apply(lambda row: row[H_col] if (row[LE_QC_flag_col]==0) & (row[H_QC_flag_col]==0) else np.nan, axis=1)
USUC2_df[H_col] = USUC2_df.apply(lambda row: row[H_col] if (row[LE_QC_flag_col]==0) & (row[H_QC_flag_col]==0) else np.nan, axis=1)
USHWB_df[H_col] = USHWB_df.apply(lambda row: row[H_col] if (row[LE_QC_flag_col]==0) & (row[H_QC_flag_col]==0) else np.nan, axis=1)

# filter negative LE during daylight
USUC1_df[LE_col] = USUC1_df.apply(lambda row: np.nan if (row[Rn_col]>0) & (row[LE_col]<0) else row[LE_col], axis=1)
USUC2_df[LE_col] = USUC2_df.apply(lambda row: np.nan if (row[Rn_col]>0) & (row[LE_col]<0) else row[LE_col], axis=1)
USHWB_df[LE_col] = USHWB_df.apply(lambda row: np.nan if (row[Rn_col]>0) & (row[LE_col]<0) else row[LE_col], axis=1)

# now divide the data into growing seasons and only keep the growing season period with 15 days of buffer for energy balance correction steps
# Define the time windows for growing seasons
US_UC_growing_season_2019 = [('2019-05-20', '2019-09-11')]
US_UC_growing_season_2020 = [('2020-05-27', '2020-09-25')]
US_UC_growing_season_2021 = [('2021-05-20', '2021-11-16')]
US_UC_growing_season_2022 = [('2022-06-08', '2022-12-08')]
US_UC_growing_season_2024 = [('2024-05-25', '2024-10-08')] 
US_HWB_growing_season_2017 = [('2017-04-15', '2017-09-30')]

def filter_time_periods(df, time_periods, buffer_days=15):
    buffer = pd.Timedelta(days=buffer_days)
    return df.loc[pd.Timestamp(time_periods[0][0]) - buffer : pd.Timestamp(time_periods[0][1]) + buffer]

USUC1_df_LE_filtered_2019 = filter_time_periods(USUC1_df, US_UC_growing_season_2019)
USUC1_df_LE_filtered_2020 = filter_time_periods(USUC1_df, US_UC_growing_season_2020)
USUC1_df_LE_filtered_2021 = filter_time_periods(USUC1_df, US_UC_growing_season_2021)
USUC1_df_LE_filtered_2022 = filter_time_periods(USUC1_df, US_UC_growing_season_2022)
USUC1_df_LE_filtered_2024 = filter_time_periods(USUC1_df, US_UC_growing_season_2024)

USUC2_df_LE_filtered_2019 = filter_time_periods(USUC2_df, US_UC_growing_season_2019)
USUC2_df_LE_filtered_2020 = filter_time_periods(USUC2_df, US_UC_growing_season_2020)
USUC2_df_LE_filtered_2021 = filter_time_periods(USUC2_df, US_UC_growing_season_2021)
USUC2_df_LE_filtered_2022 = filter_time_periods(USUC2_df, US_UC_growing_season_2022)
USUC2_df_LE_filtered_2024 = filter_time_periods(USUC2_df, US_UC_growing_season_2024)

USHWB_df_LE_filtered_2017 = filter_time_periods(USHWB_df, US_HWB_growing_season_2017)


# Replace all -9999 with np.nan values
USUC1_df_LE_filtered_2019.replace(-9999, np.nan, inplace=True)
USUC1_df_LE_filtered_2020.replace(-9999, np.nan, inplace=True)
USUC1_df_LE_filtered_2021.replace(-9999, np.nan, inplace=True)
USUC1_df_LE_filtered_2022.replace(-9999, np.nan, inplace=True)
USUC1_df_LE_filtered_2024.replace(-9999, np.nan, inplace=True)

USUC2_df_LE_filtered_2019.replace(-9999, np.nan, inplace=True)
USUC2_df_LE_filtered_2020.replace(-9999, np.nan, inplace=True)
USUC2_df_LE_filtered_2021.replace(-9999, np.nan, inplace=True)
USUC2_df_LE_filtered_2022.replace(-9999, np.nan, inplace=True)
USUC2_df_LE_filtered_2024.replace(-9999, np.nan, inplace=True)

USHWB_df_LE_filtered_2017.replace(-9999, np.nan, inplace=True)


C:\Users\adadkhah\AppData\Local\Temp\ipykernel_17324\4133567684.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  USUC1_df_LE_filtered_2019.replace(-9999, np.nan, inplace=True)
C:\Users\adadkhah\AppData\Local\Temp\ipykernel_17324\4133567684.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  USUC1_df_LE_filtered_2020.replace(-9999, np.nan, inplace=True)
C:\Users\adadkhah\AppData\Local\Temp\ipykernel_17324\4133567684.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

#### Half Hourly EB correction

In [11]:
# perform energy balance correction on half hourly values of LE and H
USUC1_df_2019_LE_H_Corrected = EBC_half_hourly_data(USUC1_df_LE_filtered_2019,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})
USUC1_df_2020_LE_H_Corrected = EBC_half_hourly_data(USUC1_df_LE_filtered_2020,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})
USUC1_df_2021_LE_H_Corrected = EBC_half_hourly_data(USUC1_df_LE_filtered_2021,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})
USUC1_df_2022_LE_H_Corrected = EBC_half_hourly_data(USUC1_df_LE_filtered_2022,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})

USUC1_df_2024_LE_H_Corrected = EBC_half_hourly_data(USUC1_df_LE_filtered_2024,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})

USUC2_df_2019_LE_H_Corrected = EBC_half_hourly_data(USUC2_df_LE_filtered_2019,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})
USUC2_df_2020_LE_H_Corrected = EBC_half_hourly_data(USUC2_df_LE_filtered_2020,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})
USUC2_df_2021_LE_H_Corrected = EBC_half_hourly_data(USUC2_df_LE_filtered_2021,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})
USUC2_df_2022_LE_H_Corrected = EBC_half_hourly_data(USUC2_df_LE_filtered_2022,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})
USUC2_df_2024_LE_H_Corrected = EBC_half_hourly_data(USUC2_df_LE_filtered_2024,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})

USHWB_df_2017_LE_H_Corrected = EBC_half_hourly_data(USHWB_df_LE_filtered_2017,
                                                       flux_cols_dict={'Rn':'NETRAD_1_1_1', 'G':'G_1_1_1','LE':'LE', 'H':'H'})


# only keep daylight hours based on Rn>0 (6 am to 6:30 pm based on local time)
USUC1_df_LE_filtered_2019_daylight = USUC1_df_2019_LE_H_Corrected.between_time("06:00", "18:30")
USUC1_df_LE_filtered_2020_daylight = USUC1_df_2020_LE_H_Corrected.between_time("06:00", "18:30")
USUC1_df_LE_filtered_2021_daylight = USUC1_df_2021_LE_H_Corrected.between_time("06:00", "18:30")
USUC1_df_LE_filtered_2022_daylight = USUC1_df_2022_LE_H_Corrected.between_time("06:00", "18:30")
USUC1_df_LE_filtered_2024_daylight = USUC1_df_2024_LE_H_Corrected.between_time("06:00", "18:30")

USUC2_df_LE_filtered_2019_daylight = USUC2_df_2019_LE_H_Corrected.between_time("06:00", "18:30")
USUC2_df_LE_filtered_2020_daylight = USUC2_df_2020_LE_H_Corrected.between_time("06:00", "18:30")
USUC2_df_LE_filtered_2021_daylight = USUC2_df_2021_LE_H_Corrected.between_time("06:00", "18:30")
USUC2_df_LE_filtered_2022_daylight = USUC2_df_2022_LE_H_Corrected.between_time("06:00", "18:30")
USUC2_df_LE_filtered_2024_daylight = USUC2_df_2024_LE_H_Corrected.between_time("06:00", "18:30")

USHWB_df_LE_filtered_2017_daylight = USHWB_df_2017_LE_H_Corrected.between_time("06:00", "18:30")

In [17]:
# aggregate data to hourly values and only keep columns that are helpful for evapotranspiration calculation/prediction
agg_funcs = {
    'LE_corr':'mean',
    'TA_1_1_1':'mean',
    'RH_1_1_1':'mean',
    'SW_IN_1_1_1':'mean',
    'LW_IN_1_1_1':'mean',
    'WS':'mean',
    'PA':'mean',
    'P_RAIN_1_1_1':'sum'   
}

# only keep columns that are going to be used for the model training
Columns_to_keep = ['LE_corr', 'TA_1_1_1', 'RH_1_1_1', 'SW_IN_1_1_1',  'LW_IN_1_1_1','WS', 'PA', 'P_RAIN_1_1_1']

USUC1_df_processed_2019 = USUC1_df_LE_filtered_2019_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC1_df_processed_2020 = USUC1_df_LE_filtered_2020_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC1_df_processed_2021 = USUC1_df_LE_filtered_2021_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC1_df_processed_2022 = USUC1_df_LE_filtered_2022_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC1_df_processed_2024 = USUC1_df_LE_filtered_2024_daylight[Columns_to_keep].resample('h').agg(agg_funcs)

USUC2_df_processed_2019 = USUC2_df_LE_filtered_2019_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC2_df_processed_2020 = USUC2_df_LE_filtered_2020_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC2_df_processed_2021 = USUC2_df_LE_filtered_2021_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC2_df_processed_2022 = USUC2_df_LE_filtered_2022_daylight[Columns_to_keep].resample('h').agg(agg_funcs)
USUC2_df_processed_2024 = USUC2_df_LE_filtered_2024_daylight[Columns_to_keep].resample('h').agg(agg_funcs)

USHWB_df_processed_2017 = USHWB_df_LE_filtered_2017_daylight[Columns_to_keep].resample('h').agg(agg_funcs)


# Now calculate ETa following the method of Harrison, L.P. 1963
TA_col = 'TA_1_1_1' # air temperature column
LE_col = 'LE_corr' # latent heat flux column

def calculate_hourly_ETa(processed_df, LE_column=LE_col, TA_column=TA_col):
    df = copy.deepcopy(processed_df)
    df['ETa_corr'] = (df[LE_col] * 60 * 60) / ((2.501 - 0.002361 * df[TA_col]) * 10**6)
    return df.drop([LE_col], axis=1)

USUC1_df_processed_2019_ETa = calculate_hourly_ETa(USUC1_df_processed_2019)
USUC1_df_processed_2020_ETa = calculate_hourly_ETa(USUC1_df_processed_2020)
USUC1_df_processed_2021_ETa = calculate_hourly_ETa(USUC1_df_processed_2021)
USUC1_df_processed_2022_ETa = calculate_hourly_ETa(USUC1_df_processed_2022)
USUC1_df_processed_2024_ETa = calculate_hourly_ETa(USUC1_df_processed_2024)

USUC2_df_processed_2019_ETa = calculate_hourly_ETa(USUC2_df_processed_2019)
USUC2_df_processed_2020_ETa = calculate_hourly_ETa(USUC2_df_processed_2020)
USUC2_df_processed_2021_ETa = calculate_hourly_ETa(USUC2_df_processed_2021)
USUC2_df_processed_2022_ETa = calculate_hourly_ETa(USUC2_df_processed_2022)
USUC2_df_processed_2024_ETa = calculate_hourly_ETa(USUC2_df_processed_2024)

USHWB_df_processed_2017_ETa = calculate_hourly_ETa(USHWB_df_processed_2017)

# filter dates to only include growing seasons
def filter_time_periods(df, time_periods, buffer_days=0):
    buffer = pd.Timedelta(days=buffer_days)
    return df.loc[pd.Timestamp(time_periods[0][0]) - buffer : pd.Timestamp(time_periods[0][1]) + buffer]

USUC1_df_FilteredDates_2019_ETa = filter_time_periods(USUC1_df_processed_2019_ETa, US_UC_growing_season_2019).between_time("06:00", "18:00")
USUC1_df_FilteredDates_2020_ETa = filter_time_periods(USUC1_df_processed_2020_ETa, US_UC_growing_season_2020).between_time("06:00", "18:00")
USUC1_df_FilteredDates_2021_ETa = filter_time_periods(USUC1_df_processed_2021_ETa, US_UC_growing_season_2021).between_time("06:00", "18:00")
USUC1_df_FilteredDates_2022_ETa = filter_time_periods(USUC1_df_processed_2022_ETa, US_UC_growing_season_2022).between_time("06:00", "18:00")
USUC1_df_FilteredDates_2024_ETa = filter_time_periods(USUC1_df_processed_2024_ETa, US_UC_growing_season_2024).between_time("06:00", "18:00")

USUC2_df_FilteredDates_2019_ETa = filter_time_periods(USUC2_df_processed_2019_ETa, US_UC_growing_season_2019).between_time("06:00", "18:00")
USUC2_df_FilteredDates_2020_ETa = filter_time_periods(USUC2_df_processed_2020_ETa, US_UC_growing_season_2020).between_time("06:00", "18:00")
USUC2_df_FilteredDates_2021_ETa = filter_time_periods(USUC2_df_processed_2021_ETa, US_UC_growing_season_2021).between_time("06:00", "18:00")
USUC2_df_FilteredDates_2022_ETa = filter_time_periods(USUC2_df_processed_2022_ETa, US_UC_growing_season_2022).between_time("06:00", "18:00")
USUC2_df_FilteredDates_2024_ETa = filter_time_periods(USUC2_df_processed_2024_ETa, US_UC_growing_season_2024).between_time("06:00", "18:00")

USHWB_df_FilteredDates_2017_ETa = filter_time_periods(USHWB_df_processed_2017_ETa, US_HWB_growing_season_2017).between_time("06:00", "18:00")

In [18]:
# export the data
USUC1_df_FilteredDates_2019_ETa.to_csv('USUC1_df_2019_ETa_MET.csv')
USUC1_df_FilteredDates_2020_ETa.to_csv('USUC1_df_2020_ETa_MET.csv')
USUC1_df_FilteredDates_2021_ETa.to_csv('USUC1_df_2021_ETa_MET.csv')
USUC1_df_FilteredDates_2022_ETa.to_csv('USUC1_df_2022_ETa_MET.csv')
USUC1_df_FilteredDates_2024_ETa.to_csv('USUC1_df_2024_ETa_MET.csv')

USUC2_df_FilteredDates_2019_ETa.to_csv('USUC2_df_2019_ETa_MET.csv')
USUC2_df_FilteredDates_2020_ETa.to_csv('USUC2_df_2020_ETa_MET.csv')
USUC2_df_FilteredDates_2021_ETa.to_csv('USUC2_df_2021_ETa_MET.csv')
USUC2_df_FilteredDates_2022_ETa.to_csv('USUC2_df_2022_ETa_MET.csv')
USUC2_df_FilteredDates_2024_ETa.to_csv('USUC2_df_2024_ETa_MET.csv')

USHWB_df_FilteredDates_2017_ETa.to_csv('USHWB_df_2017_ETa_MET.csv')

In [19]:
USUC1_df_FilteredDates_2024_ETa

,TA_1_1_1,RH_1_1_1,SW_IN_1_1_1,LW_IN_1_1_1,WS,PA,P_RAIN_1_1_1,ETa_corr
TIMESTAMP_START,,,,,,,,
2024-05-25 06:00:00,12.78950,90.62105,200.71850,298.9905,0.730552,97.27110,0.0,NaN
2024-05-25 07:00:00,16.86500,75.17220,472.17500,301.7510,0.928916,97.32435,0.0,0.208709
2024-05-25 08:00:00,20.05490,63.77765,664.41800,315.9345,2.121480,97.32270,0.0,0.185662
2024-05-25 09:00:00,22.69410,53.55450,832.10050,326.3645,2.397565,97.27545,0.0,0.290137
2024-05-25 10:00:00,24.55185,49.56075,895.99800,342.3230,2.031015,97.26305,0.0,0.235336
...,...,...,...,...,...,...,...,...
2024-10-07 14:00:00,15.47765,60.00250,127.55050,361.0270,2.185330,97.39815,0.0,0.166160
2024-10-07 15:00:00,15.45550,61.27460,195.20350,345.7030,1.958875,97.38885,0.0,0.138292
2024-10-07 16:00:00,14.72385,64.41770,63.08920,347.2620,2.075280,97.39415,0.0,0.081489
